In [1]:
%matplotlib qt
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import os
import math

In [2]:
def txt_to_df(data_files):
    dfs = {}
    for filename in data_files:
        freq = filename.split('_')[1].split('Hz')[0]
        print(f"Reading {freq}, {filename}...")
        with open(filename, 'r') as file:
            content = file.read()
            content = content.replace('"', '')
        
        with open(filename, 'w') as file:
            file.write(content)

        df = pd.read_csv(filename, delimiter=',')
    
        # print("Number of NaNs in each column:")
        # print(df.isnull().sum())
        df = df.dropna()
        
        df['Error'] = df[' Error']
        df['Target angle'] = df[' Target angle']
        df.drop(columns=[' Error', ' Target angle'], inplace=True)

        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col], errors='raise')
            except ValueError as e:
                print(f"\nOh Oh! Conversion failed in column: {col}, Error: {e}\n")
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # print("Number of NaNs in each column:")
        # print(df.isnull().sum())
        df = df.dropna()
        
        dfs[freq] = df

    return dfs

In [ ]:

def sine_wave(t, A, phase, offset, freq):
    return A * np.sin(2 * np.pi * freq * t + phase) + offset


def get_rms_amps(data, freq, ax_target, ax_error):
    time_ms = data['Run time']
    time_ms = pd.to_numeric(time_ms, errors='raise')
    time_s = time_ms / 1000

    error = data['Error']
    target_angle = data['Target angle']
    
    if max(target_angle) >= 30:
        print("30")
        start_index = np.argmax(target_angle > 30)
    elif max(target_angle) >= 27:
        print('27')
        start_index = np.argmax(target_angle >= 27)
    else:
        print('25')
        start_index = np.argmax(target_angle >= 25)
    print('start', start_index, target_angle[start_index])


    angle_range = 60
    cycles = 10 if freq in [0.1, 0.01] else 5
    points_per_wave = 500
    total_points = cycles * points_per_wave
    t_perfect = np.linspace(0, cycles / freq, total_points)
    perfect_wave = (angle_range/2) * np.cos(2 * np.pi * freq * t_perfect)
    t_perfect_shifted = t_perfect + time_s[start_index]
    # plt.figure()

    ax_target.plot(time_s, target_angle, label='Target Angle')
    ax_target.plot(t_perfect_shifted, perfect_wave, label='Perfect Angle', alpha=0.8)
    # ax_target.set_xlabel('Time (s)', fontsize=6)
    # ax_target.set_ylabel('Angle (degrees)', fontsize=6)
    ax_target.set_title(f"Target Angle at {freq}Hz", fontsize=6)
    ax_target.tick_params(axis="both", labelsize=6)
    # ax_target.legend(fontsize=6, loc='lower right')

    ax_error.plot(time_s, error, label='Error')
    # ax_error.plot(time_s, error_fit, label='Error Fit')
    # ax_error.set_xlabel('Time (s)', fontsize=6)
    # ax_error.set_ylabel('Angle (degrees)', fontsize=6)
    ax_error.set_title(f"Error at {freq}Hz", fontsize=6)
    ax_error.tick_params(axis="both", labelsize=6)
    # ax_error.legend(fontsize=6, loc='lower right')

    rms_target = np.mean(target_angle**2)**0.5
    rms_error = np.mean(error**2)**0.5
    print(f"{freq}Hz: target amplitude - {rms_target}, error amplitude - {rms_error}")

    return rms_target, rms_error





In [4]:
def plot_rms(rms_targets, rms_errors, freqs):
    combined = sorted(zip(freqs, rms_targets, rms_errors))
    freqs, rms_targets, rms_errors = zip(*combined)
    print(freqs)

    fig, ax = plt.subplots()
    ax.plot(freqs, rms_targets, label='Target RMS', linestyle='-', marker='x')
    ax.plot(freqs, rms_errors, label='Error RMS', linestyle='-', marker='x')
    ax.set_xlabel('Frequency (Hz)', fontsize=10)
    ax.set_ylabel('Amplitude (degrees)', fontsize=10)
    ax.set_title('RMS Amplitude vs Frequency', fontsize=10)
    ax.tick_params(axis="both", labelsize=10)
    ax.legend(fontsize=10, loc='upper right')
    plt.show()

In [5]:
def bode_plot(gains, phases, freqs):
    sorted_tuples = sorted(zip(freqs, gains, phases))
    freqs, gains, phases = zip(*sorted_tuples)
    print(f"Freqs: {freqs}")
    print(f"Gains: {gains}")
    print(f"Phases: {phases}")
    gain_db = 20 * np.log10(gains)
    print(gain_db)
    plt.figure()

    # Magnitude plot
    plt.subplot(2, 1, 1)
    plt.semilogx(freqs, gain_db, 'x-')
    plt.title("Bode Plot")
    plt.ylabel("Magnitude (dB)")

    # Phase plot
    plt.subplot(2, 1, 2)
    plt.semilogx(freqs, phases, 'x-')
    plt.ylabel("Phase (degrees)")
    plt.xlabel("Frequency (Hz)")

    plt.tight_layout()
    plt.show()

In [6]:
def run(data_files):
    plt.close('all')
    data = txt_to_df(data_files)
    rms_targets = []
    rms_errors = []
    freqs = []
    n_freqs = len(data_files)
    print(np.ceil(n_freqs/2))
    fig, axs = plt.subplots(int(np.ceil(n_freqs/2)), 4, figsize=(12, 3 * n_freqs))

    for i, (freq, df) in enumerate(data.items()):
        # if freq == "0.07":
        print('\n', float(freq))

        row = i // 2
        col = i % 2
        if i % 2 == 1:
            col += 1
        rms_target, rms_error = get_rms_amps(df, float(freq), axs[row, col], axs[row, col+1])
        rms_targets.append(rms_target)
        rms_errors.append(rms_error)
        freqs.append(float(freq))
            # break
    plt.tight_layout()

    plot_rms(rms_targets, rms_errors, freqs)
    # bode_plot(gains, phases, freqs)

data_files = os.listdir("data")
data_files = sorted([f"data/{f}" for f in data_files])
# print(sorted(data_files))
run(data_files)

Reading 0.01, data/data_0.01Hz.txt...

Oh Oh! Conversion failed in column: Target angle, Error: Unable to parse string " 3 18.23179" at position 806

Reading 0.02, data/data_0.02Hz.txt...
Reading 0.03, data/data_0.03Hz.txt...
Reading 0.05, data/data_0.05Hz.txt...

Oh Oh! Conversion failed in column: Target angle, Error: Unable to parse string " -27.5955.19339" at position 483

Reading 0.06, data/data_0.06Hz.txt...
Reading 0.07, data/data_0.07Hz.txt...
Reading 0.08, data/data_0.08Hz.txt...
Reading 0.09, data/data_0.09Hz.txt...
Reading 0.11, data/data_0.11Hz_leftsmooth.txt...
Reading 0.12, data/data_0.12Hz_leftsmooth.txt...
Reading 0.13, data/data_0.13_leftsmooth.txt...
Reading 0.14, data/data_0.14Hz.txt...
Reading 0.15, data/data_0.15Hz_notsmooth.txt...
Reading 0.16, data/data_0.16Hz.txt...
Reading 0.17, data/data_0.17Hz.txt...
Reading 0.18, data/data_0.18Hz.txt...
Reading 0.19, data/data_0.19Hz.txt...
Reading 0.1, data/data_0.1Hz.txt...
Reading 0.2, data/data_0.2Hz.txt...
10.0

 0.01
3